<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# title

In [1]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

# リスト見たい場合
!kaggle datasets list

# train, test等ダウンロード
! kaggle competitions download house-prices-advanced-regression-techniques -p house-prices

# ダウンロードしたらここに入る大体。
!ls /content/house-prices

Download 100%.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
cp: cannot create regular file '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, s

In [2]:
# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

# リスト見たい場合
# !kaggle datasets list

# train, test等ダウンロード
! kaggle competitions download house-prices-advanced-regression-techniques -p house-prices

# ダウンロードしたらここに入る大体。
!ls /content/house-prices

  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 24.4MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 60.6MB/s]
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 62.9MB/s]
  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 53.1MB/s]
data_description.txt  sample_submission.csv  test.csv  train.csv


In [36]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# とりあえずファイルを読み込む
train = pd.read_csv("/content/house-prices/train.csv")
test = pd.read_csv("/content/house-prices/test.csv")
# 読み込んだら確認したい
# train.head()
# test.head()

# array形式で確認したい場合（省略）
# print(np.array(test))
# 省略したくない場合は下記
# print('---')
# np.set_printoptions(threshold=np.inf)
# print(np.array(test))
# train.head(3)

# ==
# 配列の大きさを確認したい。
train['WhatIsData'] = 'Train'
test['WhatIsData'] = 'Test'
test['SalePrice'] = 9999999999
alldata = pd.concat([train,test],axis=0).reset_index(drop=True)
print('The size of train is : ' + str(train.shape))
print('The size of test is : ' + str(test.shape))
# ==
# train.info()

#  ■ わかること
# データ数：1460
# [Age, Cabin, Embarked]にnullあり。
# データ型が色々あること
# ⇒ dtypes: float64(3), int64(35), object(44)
# ==
# ■欠損値の確認
# （１つ１つpickしていくのは面倒。）
# pd.get_option("display.max_rows") # 行数取得
# pd.get_option("display.max_colwidth") # 列数取得

# ＝＝
# ■作業中■
# ■表示省略させない
# 下記はx（何で？）
# pd.set_option("display.max_rows", 80)
# pd.get_option("display.max_colwidth", 80)
# 下記はo（何で？）
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
# ＝＝
pd.options.display.max_columns = None
# print(pd.options.display.max_columns)
np.set_printoptions(threshold=np.inf)
# train.isnull()
# train.isnull().sum()

# ＝＝
# ■1以上のカラム名だけ取得できる。
# iteritems()
l = []
for index, value in train.isnull().sum().iteritems():
    if value > 0:
        l.append(index)
l
# 以下、駄文
# ■null等は勝手に埋めない。カラムの意味をしっかり把握する。
# nullに意味ある場合もある（a.g.　設備の有無等）
# ｜FLGを01で表現するか、nullで表現するかは場合による。

# ['LotFrontage',
#  'Alley',
#  'MasVnrType',
#  'MasVnrArea',
#  'BsmtQual',
#  'BsmtCond',
#  'BsmtExposure',
#  'BsmtFinType1',
#  'BsmtFinType2',
#  'Electrical',
#  'FireplaceQu',
#  'GarageType',
#  'GarageYrBlt',
#  'GarageFinish',
#  'GarageQual',
#  'GarageCond',
#  'PoolQC',
#  'Fence',
#  'MiscFeature']

# ['LotFrontage,間口の広さ
#  'Alley',路地？
# Grvl    50  ？未舗装？
# Pave    41  舗装
#  'MasVnrType',？？の種類？柄か？壁？
# BrkCmn      15
# BrkFace    445
# None       864
# Stone      128
#  'MasVnrArea',？？の面積？庭？壁？壁か？
#  ⇒ 他の人の検索するとベニヤ板って書いてあるけど違うと思う。
# 0.0       861
# 1.0         2
# 11.0        1
# 14.0        1
# 16.0        7
#          ... 
# 1115.0      1
# 1129.0      1
# 1170.0      1
# 1378.0      1
# 1600.0      1
#  'BsmtQual',Bsmtの質？■Bsmtシリーズ。何これ？
#  ⇒ 評価っぽい。？劣化度合いとか？
# Ex    121
# Fa     35
# Gd    618
# TA    649
#  'BsmtCond',条件？condominium?
# 同上。
# Fa      45
# Gd      65
# Po       2
# TA    1311
#  'BsmtExposure',暴露?
#  ⇒ 向き！でもMnとかよくわからん。
# Av    221
# Gd    134
# Mn    114
# No    953
#  'BsmtFinType1',フィン？水かき？のタイプ
# QはQualityなのでは？
# ALQ    220
# BLQ    148
# GLQ    418
# LwQ     74
# Rec    133
# Unf    430
#  'BsmtFinType2',フィン？水かき？のタイプ2
#  ⇒ 「Unf」突然増えた。
# ALQ      19
# BLQ      33
# GLQ      14
# LwQ      46
# Rec      54
# Unf    1256
#  'Electrical',電気的？
# ヒューズ。契約か？
# FuseA      94
# FuseF      27
# FuseP       3
# Mix         1
# SBrkr    1334
#  'FireplaceQu',暖炉？
# Ex     24
# Fa     33
# Gd    380
# Po     20
# TA    313
#  'GarageType',ガレージ■ガレージ
# 2Types       6
# Attchd     870
# Basment     19
# BuiltIn     88
# CarPort      9
# Detchd     387
#  'GarageYrBlt',ガレージの？？
#  ⇒ 年っぽい。
# 1900.0     1
# 1906.0     1
# 1908.0     1
# 1910.0     3
# 1914.0     2
#           ..
# 2006.0    59
# 2007.0    49
# 2008.0    29
# 2009.0    21
# 2010.0     3
#  'GarageFinish',ガレージの仕上げ？
# Fin    352
# RFn    422
# Unf    605
#  'GarageQual',ガレージの質？
# Ex       3
# Fa      48
# Gd      14
# Po       3
# TA    1311
#  'GarageCond',ガレージのcondominium?
# Ex       2
# Fa      35
# Gd       9
# Po       7
# TA    1326
#  'PoolQC',プール？
# Ex    2
# Fa    2
# Gd    3
#  'Fence',フェンス？
# GdPrv     59
# GdWo      54
# MnPrv    157
# MnWw      11
#  'MiscFeature']その他の特徴
# Gar2     2
# Othr     2
# Shed    49
# TenC     1
# カラム単位のユニーク要素をカウント。
# sort_index()つけるとorder byできる。

# ！rowが多いから減らした。
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

train['LotShape'].value_counts().sort_index()
# ＝＝
# ■カラムと欠損値の内容を調べた。
# 次に
# 欠損値を埋める？
# 使う変数を選ぶ？


The size of train is : (1460, 82)
The size of test is : (1459, 82)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



IR1    484
IR2     41
IR3     10
Reg    925
Name: LotShape, dtype: int64

In [47]:
# Street
# train.head(10)

# train.isnull().sum()
# train.dropna().shape

# 1460行
# Grvl    50
# Pave    41
# その他が 1460 - (50 + 41)
# train['Alley']
# train['Alley'].value_counts().sort_index()
# # ■NG;
# # 01に置き換える前に欠損値補填なんかできるわけない。
# train['Alley'] = train['Alley'].fillna(train['Alley'].medinan())

# train['Alley'][train['Alley'] == "Grvl"] = 0
train["Alley"][train["Alley"] == "Grvl"] = 0
# ■NG;
# intとstrだとカウントしてくれない。
# .sort_index()　を外せばok
# TypeError: '<' not supported between instances of 'str' and 'int'
train['Alley'].value_counts()
# 0       50
# Pave    41
# Grvlを0に。 ⇒ Paveを1に。
train["Alley"][train["Alley"] == "Pave"] = 1
train['Alley'].value_counts()
# 0    50
# 1    41
# ■dl
# 試しにmean()を入れてみる。
train['Alley'] = train['Alley'].fillna(train['Alley'].mean())
train['Alley'].value_counts()
# 0.450549    1369
# 0.000000      50
# 1.000000      41
# 01の評価に小数点以下は馬鹿っぽい。
# 下記にしたい。
# 舗装：1、砂利：2、路地ないよ：0
# リトライ
train = pd.read_csv("/content/house-prices/train.csv")
train["Alley"][train["Alley"] == "Grvl"] = 1
train["Alley"][train["Alley"] == "Pave"] = 2
train['Alley'] = train['Alley'].fillna(0)
train['Alley'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/l

0    1369
1      50
2      41
Name: Alley, dtype: int64

In [0]:
train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,WhatIsData
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,Train
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,Train
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,Train
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,Train
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,Train
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000,Train
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000,Train
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000,Train
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900,Train
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd

In [0]:
# train.head(10)

train.index    # index取得

# カラム、リストに。
train.columns  # カラム名一括請求

# pd.Series(train, name='MSSubClass').unique()

# 存在する要素をリストに
# 「目視確認」は馬鹿
train['Utilities'].value_counts()

AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64

In [0]:
train.head(3)

# ■わかること
# 特徴量
# Name, Ticketはめんどくさそう

#  ■ やること
# 欠損値を埋める
# [Sex, Embarked]　intに変える
# ｜平均値でいいかな


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [0]:
# 置き換え
# 方法１
# train = pd.read_csv("train.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
# test = pd.read_csv("test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)

# 方法２（見やすい）
train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1
train["Embarked"][train["Embarked"] == "S"] = 0
train["Embarked"][train["Embarked"] == "C"] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2

test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1
test["Embarked"][test["Embarked"] == "S"] = 0
test["Embarked"][test["Embarked"] == "C"] = 1
test["Embarked"][test["Embarked"] == "Q"] = 2

train.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doi

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0


In [0]:
# 欠損値を埋める
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(test["Age"].median())

train["Embarked"].fillna(train.Embarked.mean(), inplace=True)
test["Embarked"].fillna(test.Embarked.mean(), inplace=True)

train.info()
train.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null float64
dtypes: float64(3), int64(5), object(4)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0


In [0]:
# Nameを変換
# 敬称をSalutationに分類する。
# Nameは不要なので消す。
combine1 = [train]

for train in combine1: 
        train['Salutation'] = train.Name.str.extract('([A-Za-z]+).', expand=False) 
for train in combine1: 
        # 敬称に置き換えます。
        train['Salutation'] = train['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
        train['Salutation'] = train['Salutation'].replace('Mlle', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Ms', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Mme', 'Mrs')
        # 消します。
        del train['Name']
        # 置き換えた敬称をintに変換します。
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 
for train in combine1: 
        train['Salutation'] = train['Salutation'].map(Salutation_mapping) 
        train['Salutation'] = train['Salutation'].fillna(0)

# ===
# Ticketが原因のエラーあったから追加した。
for train in combine1:
        train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
        train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x))
        train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'],
                                   np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0', '0'))
        train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x))
        del train['Ticket']
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 
# ===
# train.Fare[152] = train.Fare.median()
# test.Fare[152] = test.Fare.median()

# Cabinも置き換えます。同様に。
for train in combine1: 
    train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
    train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
    train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
del train['Cabin'] 
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)

train.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett
0,1,0,3,0,22.0,1,0,7.2500,0.0,0.0,3,9,0
1,2,1,1,1,38.0,1,0,71.2833,1.0,0.0,0,8,1
2,3,1,3,1,26.0,0,0,7.9250,0.0,0.0,3,16,0


In [0]:
# 新しい変数の追加。
# 家族の人数を特徴量として扱う。

# 何人家族か
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
for train in combine1:
    # 一人フラグ
    train['IsAlone'] = 0
    train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

train.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett,FamilySize,IsAlone
0,1,0,3,0,22.0,1,0,7.2500,0.0,0.0,3,9,0,2,0
1,2,1,1,1,38.0,1,0,71.2833,1.0,0.0,0,8,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,9,1,3,1,27.0,0,2,11.1333,0.0,0.0,3,6,0,3,0
9,10,1,2,1,14.0,1,0,30.0708,1.0,0.0,2,6,0,2,0


In [0]:
train_data = train.values
xs = train_data[:, 2:] # Pclass以降の変数
y  = train_data[:, 1]  # 正解データ

xs

array([[3, 0, 22.0, 1, 0, 7.25, 0.0, 0.0, 3, 9, 0, 2, 0],
       [1, 1, 38.0, 1, 0, 71.2833, 1.0, 0.0, 0, 8, 1, 2, 0],
       [3, 1, 26.0, 0, 0, 7.925, 0.0, 0.0, 3, 16, 0, 1, 1],
       [1, 1, 35.0, 1, 0, 53.1, 0.0, 0.0, 1, 6, 1, 2, 0],
       [3, 0, 35.0, 0, 0, 8.05, 0.0, 0.0, 3, 6, 0, 1, 1],
       [3, 0, 28.0, 0, 0, 8.4583, 2.0, 0.0, 3, 6, 0, 1, 1],
       [1, 0, 54.0, 0, 0, 51.8625, 0.0, 0.0, 1, 5, 2, 1, 1],
       [3, 0, 2.0, 3, 1, 21.075, 0.0, 0.0, 3, 6, 0, 5, 0],
       [3, 1, 27.0, 0, 2, 11.1333, 0.0, 0.0, 3, 6, 0, 3, 0],
       [2, 1, 14.0, 1, 0, 30.0708, 1.0, 0.0, 2, 6, 0, 2, 0],
       [3, 1, 4.0, 1, 1, 16.7, 0.0, 0.0, 0, 7, 0, 3, 0],
       [1, 1, 58.0, 0, 0, 26.55, 0.0, 0.0, 1, 6, 1, 1, 1],
       [3, 0, 20.0, 0, 0, 8.05, 0.0, 0.0, 3, 9, 0, 1, 1],
       [3, 0, 39.0, 1, 5, 31.275, 0.0, 0.0, 3, 6, 0, 7, 0],
       [3, 1, 14.0, 0, 0, 7.8542, 0.0, 0.0, 3, 6, 0, 1, 1],
       [2, 1, 55.0, 0, 0, 16.0, 0.0, 0.0, 2, 6, 0, 1, 1],
       [3, 0, 2.0, 4, 1, 29.125, 2.0, 0.0, 3, 6, 0,

In [0]:
test.info()

# ■わかること
# 欠損値

#  ■ やること
# 欠損値を埋める

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null int64
dtypes: float64(2), int64(5), object(4)
memory usage: 36.0+ KB


In [0]:
# test_shape = test.shape
# train_shape = train.shape

# print(test_shape)
# print(train_shape)

# # test.describe()
# train.describe()

# def kesson_table(df): 
#         null_val = df.isnull().sum()
#         percent = 100 * df.isnull().sum()/len(df)
#         kesson_table = pd.concat([null_val, percent], axis=1)
#         kesson_table_ren_columns = kesson_table.rename(
#         columns = {0 : '欠損数', 1 : '%'})
#         return kesson_table_ren_columns
 
# kesson_table(train)
# # kesson_table(test)

# train["Age"] = train["Age"].fillna(train["Age"].median())
# train["Embarked"] = train["Embarked"].fillna("S")
# kesson_table(train)

# train.head(10)

# # test["Age"] = test["Age"].fillna(test["Age"].median())
# # test["Sex"][test["Sex"] == "male"] = 0
# # test["Sex"][test["Sex"] == "female"] = 1
# # test["Embarked"][test["Embarked"] == "S"] = 0
# # test["Embarked"][test["Embarked"] == "C"] = 1
# # test["Embarked"][test["Embarked"] == "Q"] = 2
# # test.Fare[152] = test.Fare.median()


# # train["Age"] = train["Age"].fillna(train["Age"].median())
# # train["Sex"][train["Sex"] == "male"] = 0
# # train["Sex"][train["Sex"] == "female"] = 1
# # train["Embarked"][train["Embarked"] == "S"] = 0
# # train["Embarked"][train["Embarked"] == "C"] = 1
# # train["Embarked"][train["Embarked"] == "Q"] = 2
# # train.Fare[152] = train.Fare.median()

# # test.head(10)
# train.head(10)

(418, 11)
(891, 12)


/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0
5,6,0,3,"Moran, Mr. James",0,28.0,0,0,330877,8.4583,NaN,2
6,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,E46,0
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2.0,3,1,349909,21.0750,NaN,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,0,2,347742,11.1333,NaN,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,NaN,1


In [0]:
# testも同じ処理を行う。
test["Age"].fillna(train.Age.mean(), inplace=True)
test["Fare"].fillna(train.Fare.mean(), inplace=True)

combine = [test]
for test in combine:
    test['Salutation'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
for test in combine:
    test['Salutation'] = test['Salutation'].replace(['Lady', 'Countess','Capt', 'Col',\
         'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    test['Salutation'] = test['Salutation'].replace('Mlle', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Ms', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Mme', 'Mrs')
    del test['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for test in combine:
    test['Salutation'] = test['Salutation'].map(Salutation_mapping)
    test['Salutation'] = test['Salutation'].fillna(0)

for test in combine:
        test['Ticket_Lett'] = test['Ticket'].apply(lambda x: str(x)[0])
        test['Ticket_Lett'] = test['Ticket_Lett'].apply(lambda x: str(x))
        test['Ticket_Lett'] = np.where((test['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), test['Ticket_Lett'],
                                   np.where((test['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0', '0'))
        test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
        del test['Ticket']
test['Ticket_Lett']=test['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 

for test in combine:
        test['Cabin_Lett'] = test['Cabin'].apply(lambda x: str(x)[0])
        test['Cabin_Lett'] = test['Cabin_Lett'].apply(lambda x: str(x))
        test['Cabin_Lett'] = np.where((test['Cabin_Lett']).isin(['T', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']),test['Cabin_Lett'],
                                   np.where((test['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0','0'))        
        del test['Cabin']
test['Cabin_Lett']=test['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1).replace("G",1) 

test["FamilySize"] = train["SibSp"] + train["Parch"] + 1

for test in combine:
    test['IsAlone'] = 0
    test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1
    
test_data = test.values
xs_test = test_data[:, 1:]

test.head(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett,FamilySize,IsAlone
0,892,3,0,34.5,0,0,7.8292,2,1,3,6,0,2,0
1,893,3,1,47.0,1,0,7.0000,0,3,3,6,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,900,3,1,18.0,0,0,7.2292,1,3,2,4,0,3,0
9,901,3,0,21.0,2,0,24.1500,0,1,3,9,0,2,0


In [0]:
# test.head(10)
# train.head(10)

# xs_test
# 全部表示
np.set_printoptions(threshold=np.inf)
xs
# y

# 問題

array([[3, 0, 22.0, 1, 0, 7.25, 0.0, 0.0, 3, 9, 0, 2, 0],
       [1, 1, 38.0, 1, 0, 71.2833, 1.0, 0.0, 0, 8, 1, 2, 0],
       [3, 1, 26.0, 0, 0, 7.925, 0.0, 0.0, 3, 16, 0, 1, 1],
       [1, 1, 35.0, 1, 0, 53.1, 0.0, 0.0, 1, 6, 1, 2, 0],
       [3, 0, 35.0, 0, 0, 8.05, 0.0, 0.0, 3, 6, 0, 1, 1],
       [3, 0, 28.0, 0, 0, 8.4583, 2.0, 0.0, 3, 6, 0, 1, 1],
       [1, 0, 54.0, 0, 0, 51.8625, 0.0, 0.0, 1, 5, 2, 1, 1],
       [3, 0, 2.0, 3, 1, 21.075, 0.0, 0.0, 3, 6, 0, 5, 0],
       [3, 1, 27.0, 0, 2, 11.1333, 0.0, 0.0, 3, 6, 0, 3, 0],
       [2, 1, 14.0, 1, 0, 30.0708, 1.0, 0.0, 2, 6, 0, 2, 0],
       [3, 1, 4.0, 1, 1, 16.7, 0.0, 0.0, 0, 7, 0, 3, 0],
       [1, 1, 58.0, 0, 0, 26.55, 0.0, 0.0, 1, 6, 1, 1, 1],
       [3, 0, 20.0, 0, 0, 8.05, 0.0, 0.0, 3, 9, 0, 1, 1],
       [3, 0, 39.0, 1, 5, 31.275, 0.0, 0.0, 3, 6, 0, 7, 0],
       [3, 1, 14.0, 0, 0, 7.8542, 0.0, 0.0, 3, 6, 0, 1, 1],
       [2, 1, 55.0, 0, 0, 16.0, 0.0, 0.0, 2, 6, 0, 1, 1],
       [3, 0, 2.0, 4, 1, 29.125, 2.0, 0.0, 3, 6, 0,

In [0]:
# 3/5 前処理が終わったので学習
# これはエラー出る。（後学のため残す。
from sklearn.ensemble import RandomForestClassifier

# y=y.astype('int')
# y = train[:, 1]

random_forest=RandomForestClassifier()
random_forest.fit(xs, y)
Y_pred = random_forest.predict(xs_test)

import csv
with open("predict_result_data.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test_data[:,0].astype(int), Y_pred.astype(int)):
        writer.writerow([pid, survived])

# １＿Ticketを処理しないと出力されるエラー
# 対処：Ticketを処置したら直った。
# ＝＝
# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-174-9578c4bce77f> in <module>()
#       2 
#       3 random_forest=RandomForestClassifier()
# ----> 4 random_forest.fit(xs, y)
#       5 Y_pred = random_forest.predict(xs_test)
#       6 

# 2 frames
# /usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py in asarray(a, dtype, order)
#      83 
#      84     """
# ---> 85     return array(a, dtype, copy=False, order=order)
#      86 
#      87 

# ValueError: could not convert string to float: 'A/5 21171'
# ＝＝        

# ２＿なんかよくわからんエラー
# ＝＝
# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-185-1f83660e0be7> in <module>()
#       2 
#       3 random_forest=RandomForestClassifier()
# ----> 4 random_forest.fit(xs, y)
#       5 Y_pred = random_forest.predict(xs_test)
#       6 

# 2 frames
# /usr/local/lib/python3.6/dist-packages/sklearn/utils/multiclass.py in check_classification_targets(y)
#     167     if y_type not in ['binary', 'multiclass', 'multiclass-multioutput',
#     168                       'multilabel-indicator', 'multilabel-sequences']:
# --> 169         raise ValueError("Unknown label type: %r" % y_type)
#     170 
#     171 

# ValueError: Unknown label type: 'unknown'
# ＝＝

In [0]:
# test.head(10)
# train.head(10)

# xs_test
# 全部表示
# np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
# xs
# y

# 問題
# train　Ticketが変換されてない

train
# for train in combine:
#         train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
#         train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x))
#         train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'],
#                                    np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
#                                             '0', '0'))
#         train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x))
#         del train['Ticket']
# train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 

KeyError: ignored

In [0]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn import grid_search
from sklearn.model_selection import GridSearchCV
# from sklearn.grid_search import GridSearchCV

"""
parameters = {
        'n_estimators'      : [10,25,50,75,100],
        'random_state'      : [0],
        'n_jobs'            : [4],
        'min_samples_split' : [5,10, 15, 20,25, 30],
        'max_depth'         : [5, 10, 15,20,25,30]
}

clf = grid_search.GridSearchCV(RandomForestClassifier(), parameters)
clf.fit(xs, y)
 
print(clf.best_estimator_)
"""

"\nparameters = {\n        'n_estimators'      : [10,25,50,75,100],\n        'random_state'      : [0],\n        'n_jobs'            : [4],\n        'min_samples_split' : [5,10, 15, 20,25, 30],\n        'max_depth'         : [5, 10, 15,20,25,30]\n}\n\nclf = grid_search.GridSearchCV(RandomForestClassifier(), parameters)\nclf.fit(xs, y)\n \nprint(clf.best_estimator_)\n"

In [0]:
# まとめ
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
train= pd.read_csv("train.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
test= pd.read_csv("test.csv").replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2) 
train["Age"].fillna(train.Age.mean(), inplace=True) 
train["Embarked"].fillna(train.Embarked.mean(), inplace=True) 
combine1 = [train]

for train in combine1: 
        train['Salutation'] = train.Name.str.extract(' ([A-Za-z]+).', expand=False) 
for train in combine1: 
        train['Salutation'] = train['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
        train['Salutation'] = train['Salutation'].replace('Mlle', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Ms', 'Miss')
        train['Salutation'] = train['Salutation'].replace('Mme', 'Mrs')
        del train['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 

for train in combine1: 
        train['Salutation'] = train['Salutation'].map(Salutation_mapping) 
        train['Salutation'] = train['Salutation'].fillna(0) 
    
for train in combine1: 
        train['Ticket_Lett'] = train['Ticket'].apply(lambda x: str(x)[0])
        train['Ticket_Lett'] = train['Ticket_Lett'].apply(lambda x: str(x)) 
        train['Ticket_Lett'] = np.where((train['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), train['Ticket_Lett'], np.where((train['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')) 
        train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x)) 
        del train['Ticket'] 
train['Ticket_Lett']=train['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 

    
for train in combine1: 
    train['Cabin_Lett'] = train['Cabin'].apply(lambda x: str(x)[0]) 
    train['Cabin_Lett'] = train['Cabin_Lett'].apply(lambda x: str(x)) 
    train['Cabin_Lett'] = np.where((train['Cabin_Lett']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']),train['Cabin_Lett'], np.where((train['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0'))
del train['Cabin'] 
train['Cabin_Lett']=train['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1) 
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
for train in combine1:
    train['IsAlone'] = 0
    train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1
train_data = train.values
xs = train_data[:, 2:] # Pclass以降の変数
y  = train_data[:, 1]  # 正解データ
test["Age"].fillna(train.Age.mean(), inplace=True)
test["Fare"].fillna(train.Fare.mean(), inplace=True)

combine = [test]
for test in combine:
    test['Salutation'] = test.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
for test in combine:
    test['Salutation'] = test['Salutation'].replace(['Lady', 'Countess','Capt', 'Col',\
         'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    test['Salutation'] = test['Salutation'].replace('Mlle', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Ms', 'Miss')
    test['Salutation'] = test['Salutation'].replace('Mme', 'Mrs')
    del test['Name']
Salutation_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for test in combine:
    test['Salutation'] = test['Salutation'].map(Salutation_mapping)
    test['Salutation'] = test['Salutation'].fillna(0)

for test in combine:
        test['Ticket_Lett'] = test['Ticket'].apply(lambda x: str(x)[0])
        test['Ticket_Lett'] = test['Ticket_Lett'].apply(lambda x: str(x))
        test['Ticket_Lett'] = np.where((test['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), test['Ticket_Lett'],
                                   np.where((test['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0', '0'))
        test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
        del test['Ticket']
test['Ticket_Lett']=test['Ticket_Lett'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3) 

for test in combine:
        test['Cabin_Lett'] = test['Cabin'].apply(lambda x: str(x)[0])
        test['Cabin_Lett'] = test['Cabin_Lett'].apply(lambda x: str(x))
        test['Cabin_Lett'] = np.where((test['Cabin_Lett']).isin(['T', 'H', 'G', 'F', 'E', 'D', 'C', 'B', 'A']),test['Cabin_Lett'],
                                   np.where((test['Cabin_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            '0','0'))        
        del test['Cabin']
test['Cabin_Lett']=test['Cabin_Lett'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1).replace("G",1) 
test["FamilySize"] = train["SibSp"] + train["Parch"] + 1

for test in combine:
    test['IsAlone'] = 0
    test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1
    
test_data = test.values
xs_test = test_data[:, 1:]
from sklearn.ensemble import RandomForestClassifier

random_forest=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=51, n_jobs=4,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

random_forest.fit(xs, y)
Y_pred = random_forest.predict(xs_test)
"""random_forest.score(xs,y)
acc_random_forest = round(random_forest.score(xs,y) * 100, 2)
acc_random_forest
"""

import csv
with open("predict_result_data5.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["PassengerId", "Survived"])
    for pid, survived in zip(test_data[:,0].astype(int), Y_pred.astype(int)):
        writer.writerow([pid, survived])

In [0]:
# 表示設定
# np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 5)
train

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salutation,Ticket_Lett,Ticket_Len,Cabin_Lett,FamilySize,IsAlone
0,892,3,0,34.5,0,0,7.8292,2,1,3,6,0,2,0
1,893,3,1,47.0,1,0,7.0000,0,3,3,6,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,1308,3,0,27.0,0,0,8.0500,0,1,3,6,0,3,0
417,1309,3,0,27.0,1,1,22.3583,1,4,2,4,0,3,0


In [0]:
# 以下、他の。
test_shape = test.shape
train_shape = train.shape

print(test_shape)
print(train_shape)

# test.describe()
train.describe()

def kesson_table(df): 
        null_val = df.isnull().sum()
        percent = 100 * df.isnull().sum()/len(df)
        kesson_table = pd.concat([null_val, percent], axis=1)
        kesson_table_ren_columns = kesson_table.rename(
        columns = {0 : '欠損数', 1 : '%'})
        return kesson_table_ren_columns
 
kesson_table(train)
# kesson_table(test)

train["Age"] = train["Age"].fillna(train["Age"].median())
train["Embarked"] = train["Embarked"].fillna("S")
kesson_table(train)

train.head(10)

test["Age"] = test["Age"].fillna(test["Age"].median())
test["Sex"][test["Sex"] == "male"] = 0
test["Sex"][test["Sex"] == "female"] = 1
test["Embarked"][test["Embarked"] == "S"] = 0
test["Embarked"][test["Embarked"] == "C"] = 1
test["Embarked"][test["Embarked"] == "Q"] = 2
test.Fare[152] = test.Fare.median()


train["Age"] = train["Age"].fillna(train["Age"].median())
train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1
train["Embarked"][train["Embarked"] == "S"] = 0
train["Embarked"][train["Embarked"] == "C"] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2
train.Fare[152] = train.Fare.median()

# test.head(10)
train.head(10)

(418, 11)
(891, 12)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be s

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0
5,6,0,3,"Moran, Mr. James",0,28.0,0,0,330877,8.4583,NaN,2
6,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,E46,0
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2.0,3,1,349909,21.0750,NaN,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,0,2,347742,11.1333,NaN,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,NaN,1


In [0]:
from sklearn import tree

# PassengerIdを取得
PassengerId = np.array(test["PassengerId"]).astype(int)
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
# my_tree_one.csvとして書き出し
my_solution.to_csv("my_tree_one.csv", index_label = ["PassengerId"])

# 「train」の目的変数と説明変数の値を取得
target = train["Survived"].values
features_one = train[["Pclass",  "Sex", "Age", "Fare"]].values
# 決定木の作成
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)
# 「test」の説明変数の値を取得
test_features = test[["Pclass", "Sex",  "Age", "Fare"]].values
# 「test」の説明変数を使って「my_tree_one」のモデルで予測
my_prediction = my_tree_one.predict(test_features)


NameError: ignored

In [0]:
my_prediction.shape

print(my_prediction)

# PassengerIdを取得
PassengerId = np.array(test["PassengerId"]).astype(int)
# my_prediction(予測データ）とPassengerIdをデータフレームへ落とし込む
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
# my_tree_one.csvとして書き出し
my_solution.to_csv("my_tree_one.csv", index_label = ["PassengerId"])

In [0]:
!df -h